# SteamMonitor
Python script to monitor the status of a user.

## Requirements
- Python 3.x or higher. [Download](https://www.python.org/downloads/)
- Jupyter if you want to use .ipynb version
    ```bash
    pip install jupyter
    ```

## Install
1. Clone the repository by git
    ```bash
    git clone https://github.com/Cryptinum/SteamMonitor.git
    ```

2. Install requirements by pip
    ```bash
    pip install -r requirements.txt
    ```

## Run
- Add python to environment variables (usually auto added after install Python). Run the line below in cmd in the script path.
    ```bash
    python SteamMonitor.py
    ```

## Configuration
- Change `steamID` to the ID of a user you want to monitor, the status of the user should be public.
- Change `period` to the time period you want to monitor, its unit is minute.

## Note
- There is also a .ipynb version for Jupyter user.
- This script will create two logs, one is for save status for every minutes, another is for save status when the user status changes. Both logs are in .csv format.
- Not recommended to change period less than 1, as it may lead to request failure.

## Disclaimer
- Only for educational purpose. Do not use it for illegal activities.

In [17]:
import warnings, requests, time
from bs4 import BeautifulSoup

# import pandas as pd
# import numpy as np
from datetime import datetime

# Global Settings
# np.set_printoptions(suppress=True)
warnings.filterwarnings("ignore")  # Suppress warnings.
# pd.options.display.max_rows = 10  # Display no more than 10 rows.

In [19]:
def forteenDaysTotal(soup):
    """Total game time in last 14 days, 0 if fail to get element."""
    source = soup.find_all("div", class_="recentgame_quicklinks recentgame_recentplaytime")
    return float(source[0].text.strip()[:-19]) if source else 0.0

def statusNow(soup):
    """Current status."""
    return soup.find_all("div", class_="profile_in_game_header")[0].text.strip()

def gameNow(soup):
    """Current game, empty string if fail to get element (no game playing)."""
    source = soup.find_all("div", class_="profile_in_game_name")
    return source[0].text.strip() if source else ""

In [ ]:
# User for monitoring
steamID = "M3DJH"
period = 1


urlTemplate = "https://steamcommunity.com/id/{}"
url = urlTemplate.format(steamID)
headers = {
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"
}

previousStatus = [None for _ in range(4)]

while True:
    try:
        # Request user page
        page = requests.get(url, headers=headers, verify=False)  # Do not verify
        page.encoding = "utf-8"
        soup = BeautifulSoup(page.text, "html.parser")

        if page.status_code is 200:
            now = datetime.now().strftime("%Y-%m-%d %H:%M")
            total = forteenDaysTotal(soup)
            status = statusNow(soup)
            game = gameNow(soup)
            nowStatus = [total, status, game]

            with open(steamID+'_StatusLog.csv','a') as file:
                file.write(f'{now},{total},{status},{game}\n')
            if nowStatus != previousStatus:
                with open(steamID+'_ChangeLog.csv', 'a') as changeFile:
                    changeFile.write(f'{now},{total},{status},{game}\n')
                print(f'[{now}],{total},{status} - {game}')
                previousStatus = nowStatus
            else:
                print(f'[{now}],{total},{status} - {game}')
        else:
            print(f'Failed to retrieve data for {steamID}. Status Code: {page.status_code}')

    except requests.exceptions.RequestException as e:
        print(f"Error occurred for {steamID}: {e}")
    time.sleep(period * 60)